<a href="https://colab.research.google.com/github/NamishBansal15/transformer-modeling-25/blob/main/EfficientDetREACTORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -----------------------------------------------
# Fast EfficientDet-style Substation Detector - Fixed Version
# -----------------------------------------------

import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import os
import json
from sklearn.utils import shuffle

# ----------------------------
# CONFIGURATION
# ----------------------------
CONFIG = {
    'model_name': 'efficientnet-b0',
    'num_classes': 3,  # reactor, transformer, circuit_breaker
    'image_size': 224,
    'batch_size': 16,
    'epochs': 50,
    'learning_rate': 0.001,
    'train_data_path': '/root/',
    'val_data_path': '/mnt/',
    'test_data_path': '/boot/',
    'train_annotation_file': '/root/_annotations.coco.json',
    'val_annotation_file': '/mnt/_annotations.coco.json',
    'test_annotation_file': '/boot/_annotations.coco.json',
    'max_objects_per_image': 5
}

# ----------------------------
# Data Generator
# ----------------------------
class OptimizedSubstationDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_data, batch_size, image_size, num_classes, max_objects):
        self.image_data = image_data
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.max_objects = max_objects
        self.indexes = np.arange(len(image_data))
        self.on_epoch_end()

    def __len__(self):
        return max(1, len(self.image_data) // self.batch_size)

    def on_epoch_end(self):
        self.indexes = shuffle(self.indexes)

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        if len(batch_indexes) == 0:
            batch_indexes = [0]

        batch_images = []
        batch_classifications = []
        batch_regressions = []

        for i in batch_indexes:
            if i >= len(self.image_data):
                i = 0

            image_info = self.image_data[i]
            image = self.load_image(image_info['filename'], image_info['dataset_type'])
            labels = self.load_annotations(image_info)

            batch_images.append(image)

            if labels['classes'] and labels['boxes']:
                dominant_class = max(set(labels['classes']), key=labels['classes'].count)
                boxes = np.array(labels['boxes'])
                # Calculate the center and dimensions of the bounding box
                x_center = (boxes[:, 1] + boxes[:, 3]) / 2
                y_center = (boxes[:, 0] + boxes[:, 2]) / 2
                width = boxes[:, 3] - boxes[:, 1]
                height = boxes[:, 2] - boxes[:, 0]
                # Calculate the average center and dimensions across all boxes
                avg_box = np.array([np.mean(y_center), np.mean(x_center), np.mean(height), np.mean(width)])

                batch_classifications.append(dominant_class)
                batch_regressions.append(avg_box)
            else:
                # Default values for images with no objects
                batch_classifications.append(0)
                batch_regressions.append([0.5, 0.5, 0.2, 0.2]) # Use center and dimensions format

        return (
            np.array(batch_images),
            {
                'classification': np.array(batch_classifications),
                'regression': np.array(batch_regressions)
            }
        )

    @tf.function
    def load_image(self, image_path, dataset_type):
        if dataset_type == 'train':
            base_path = CONFIG['train_data_path']
        elif dataset_type == 'valid':
            base_path = CONFIG['val_data_path']
        elif dataset_type == 'test':
            base_path = CONFIG['test_data_path']
        else:
            print(f"Warning: Unknown dataset type {dataset_type}. Using train path.")
            base_path = CONFIG['train_data_path']

        full_path = os.path.join(base_path, image_path)
        try:
            image = tf.io.read_file(full_path)
            if full_path.lower().endswith('.png'):
                image = tf.image.decode_png(image, channels=3)
            else:
                image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, [self.image_size, self.image_size])
            image = tf.cast(image, tf.float32) / 255.0
            return image
        except Exception as e:
            print(f"Error loading image {full_path}: {e}")
            return tf.zeros((self.image_size, self.image_size, 3), dtype=tf.float32)

    def load_annotations(self, annotation):
        boxes = []
        classes = []

        if 'objects' not in annotation or not annotation['objects']:
            return {'boxes': [], 'classes': []}

        img_width = annotation.get('width', 1)
        img_height = annotation.get('height', 1)

        for obj in annotation['objects']:
            if 'bbox' not in obj:
                continue

            x, y, w, h = obj['bbox']
            if w <= 0 or h <= 0:
                continue

            x1 = max(0, x / img_width)
            y1 = max(0, y / img_height)
            x2 = min(1, (x + w) / img_width)
            y2 = min(1, (y + h) / img_height)

            if x2 <= x1 or y2 <= y1:
                continue

            boxes.append([y1, x1, y2, x2])

            class_map = {'reactor': 0, 'transformer': 1, 'circuit_breaker': 2}
            class_name = obj.get('class', obj.get('category', 'unknown'))
            class_name = class_name.strip().lower().replace(" ", "_")
            if class_name in class_map:
                classes.append(class_map[class_name])
            else:
                print(f"Warning: Unknown class '{class_name}', using default class 0")
                classes.append(0)

        return {'boxes': boxes, 'classes': classes}

# ----------------------------
# Parse COCO
# ----------------------------
def parse_coco_annotations(annotation_file, dataset_type):
    print(f"Parsing COCO annotations from {annotation_file} for {dataset_type} dataset...")
    with open(annotation_file, 'r') as f:
        coco_data = json.load(f)

    images_dict = {img['id']: img for img in coco_data['images']}
    categories_dict = {cat['id']: cat['name'] for cat in coco_data['categories']}

    image_annotations = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        if image_id not in image_annotations:
            image_annotations[image_id] = []
        image_annotations[image_id].append(ann)

    processed_data = []
    for image_id, image_info in images_dict.items():
        objects = []
        if image_id in image_annotations:
            for ann in image_annotations[image_id]:
                bbox = ann['bbox']
                if len(bbox) == 4 and bbox[2] > 0 and bbox[3] > 0:
                    obj = {
                        'class': categories_dict[ann['category_id']],
                        'bbox': bbox
                    }
                    objects.append(obj)

        processed_data.append({
            'filename': image_info['file_name'],
            'width': image_info.get('width', 1),
            'height': image_info.get('height', 1),
            'objects': objects,
            'dataset_type': dataset_type # Add dataset type information
        })

    return processed_data

# ----------------------------
# Model
# ----------------------------
def create_fast_detection_model(num_classes, image_size):
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(image_size, image_size, 3),
        weights='imagenet',
        include_top=False
    )
    base_model.trainable = False

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    class_branch = tf.keras.layers.Dense(128, activation='relu')(x)
    class_branch = tf.keras.layers.Dropout(0.2)(class_branch)
    classification_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification')(class_branch)

    reg_branch = tf.keras.layers.Dense(128, activation='relu')(x)
    reg_branch = tf.keras.layers.Dropout(0.2)(reg_branch)
    regression_output = tf.keras.layers.Dense(4, activation='sigmoid', name='regression')(reg_branch) # Outputting [y_center, x_center, height, width]

    model = tf.keras.Model(inputs=base_model.input, outputs=[classification_output, regression_output])
    return model

# ----------------------------
# Train
# ----------------------------
def train_efficientdet():
    print("Starting training...")

    # Check for annotation files
    if not os.path.exists(CONFIG['train_annotation_file']):
        print(f"Train annotation file not found at {CONFIG['train_annotation_file']}")
        return None, None, None, None
    if not os.path.exists(CONFIG['val_annotation_file']):
        print(f"Validation annotation file not found at {CONFIG['val_annotation_file']}")
        return None, None, None, None
    if not os.path.exists(CONFIG['test_annotation_file']):
        print(f"Test annotation file not found at {CONFIG['test_annotation_file']}")
        return None, None, None, None

    # Check for data directories
    if not os.path.exists(CONFIG['train_data_path']):
        print(f"Train data path not found at {CONFIG['train_data_path']}")
        return None, None, None, None
    if not os.path.exists(CONFIG['val_data_path']):
        print(f"Validation data path not found at {CONFIG['val_data_path']}")
        return None, None, None, None
    if not os.path.exists(CONFIG['test_data_path']):
        print(f"Test data path not found at {CONFIG['test_data_path']}")
        return None, None, None, None


    train_data = parse_coco_annotations(CONFIG['train_annotation_file'], 'train')
    val_data = parse_coco_annotations(CONFIG['val_annotation_file'], 'valid')
    test_data = parse_coco_annotations(CONFIG['test_annotation_file'], 'test')


    train_generator = OptimizedSubstationDataGenerator(
        train_data, CONFIG['batch_size'], CONFIG['image_size'], CONFIG['num_classes'], CONFIG['max_objects_per_image'])
    val_generator = OptimizedSubstationDataGenerator(
        val_data, CONFIG['batch_size'], CONFIG['image_size'], CONFIG['num_classes'], CONFIG['max_objects_per_image'])
    test_generator = OptimizedSubstationDataGenerator(
        test_data, CONFIG['batch_size'], CONFIG['image_size'], CONFIG['num_classes'], CONFIG['max_objects_per_image'])


    model = create_fast_detection_model(CONFIG['num_classes'], CONFIG['image_size'])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG['learning_rate']),
        loss={'classification': 'sparse_categorical_crossentropy', 'regression': 'mse'},
        loss_weights={'classification': 1.0, 'regression': 0.5},
        metrics={'classification': ['accuracy'], 'regression': ['mae']}
    )

    model.summary()

    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss'),
        ModelCheckpoint('efficientdet_substation_best.h5', save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(patience=5, factor=0.5, monitor='val_loss', min_lr=1e-7)
    ]

    history = model.fit(
        train_generator,
        epochs=CONFIG['epochs'],
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )

    model.save('efficientdet_substation_final.h5')

    print("\nFine-tuning...")
    model.layers[0].trainable = True
    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG['learning_rate'] * 0.1),
        loss={'classification': 'sparse_categorical_crossentropy', 'regression': 'mse'},
        loss_weights={'classification': 1.0, 'regression': 0.5},
        metrics={'classification': ['accuracy'], 'regression': ['mae']}
    )

    history_finetune = model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )

    model.save('efficientdet_substation_finetuned.h5')
    return model, history, val_generator, test_generator

# ----------------------------
# Evaluate
# ----------------------------
def evaluate_model(model, data_generator, dataset_name):
    print(f"\n✅ Final Evaluation on {dataset_name} Dataset:")
    results = model.evaluate(data_generator, verbose=1)
    print(f"  ➜ Total Loss: {results[0]:.4f}")
    print(f"  ➜ Classification Loss: {results[1]:.4f}")
    print(f"  ➜ Regression Loss: {results[2]:.4f}")
    print(f"  ➜ Classification Accuracy: {results[3]*100:.2f}%")
    print(f"  ➜ Regression MAE: {results[4]:.4f}")


# ----------------------------
# Run
# ----------------------------
if __name__ == "__main__":
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Using GPU: {gpus}")
        except RuntimeError as e:
            print(f"GPU setup error: {e}")
    else:
        print("No GPU found, using CPU.")


    model, history, val_generator, test_generator = train_efficientdet()

    if model and test_generator:
        evaluate_model(model, val_generator, "Validation")
        evaluate_model(model, test_generator, "Test")

Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Starting training...
Parsing COCO annotations from /root/_annotations.coco.json for train dataset...
Parsing COCO annotations from /mnt/_annotations.coco.json for valid dataset...
Parsing COCO annotations from /boot/_annotations.coco.json for test dataset...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,445,226 (16.96 MB)

 Trainable params: 395,143 (1.51 MB)

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0082 - loss: 0.0248 - regression_loss: 0.0334 - regression_mae: 0.1381 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0260 - val_loss: 0.0324 - val_regression_loss: 0.0128 - val_regression_mae: 0.0949 - learning_rate: 0.0010
Epoch 16/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0069 - loss: 0.0224 - regression_loss: 0.0310 - regression_mae: 0.1335 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0259 - val_loss: 0.0366 - val_regression_loss: 0.0215 - val_regression_mae: 0.1095 - learning_rate: 0.0010
Epoch 17/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0074 - loss: 0.0264 - regression_loss: 0.0379 - regression_mae: 0.1456 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0267 - val_loss: 0.0384 - val_regression_loss: 0.0234 


Fine-tuning...
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - classification_accuracy: 1.0000 - classification_loss: 0.0087 - loss: 0.0256 - regression_loss: 0.0339 - regression_mae: 0.1407 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0271 - val_loss: 0.0391 - val_regression_loss: 0.0240 - val_regression_mae: 0.1192 - learning_rate: 1.0000e-04
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0040 - loss: 0.0222 - regression_loss: 0.0365 - regression_mae: 0.1370 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0249 - val_loss: 0.0328 - val_regression_loss: 0.0156 - val_regression_mae: 0.1020 - learning_rate: 1.0000e-04
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0062 - loss: 0.0240 - regression_loss: 0.0356 - regression_mae: 0.1463

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0059 - loss: 0.0236 - regression_loss: 0.0354 - regression_mae: 0.1458 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0231 - val_loss: 0.0299 - val_regression_loss: 0.0137 - val_regression_mae: 0.0978 - learning_rate: 1.0000e-04
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0042 - loss: 0.0210 - regression_loss: 0.0336 - regression_mae: 0.1395 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0219 - val_loss: 0.0337 - val_regression_loss: 0.0235 - val_regression_mae: 0.1144 - learning_rate: 1.0000e-04
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0064 - loss: 0.0203 - regression_loss: 0.0279 - regression_mae: 0.1310 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0203 - val_loss: 0.0300 - val_regression_loss: 0.

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 287ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0054 - loss: 0.0216 - regression_loss: 0.0325 - regression_mae: 0.1344 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0193 - val_loss: 0.0266 - val_regression_loss: 0.0145 - val_regression_mae: 0.1001 - learning_rate: 1.0000e-04
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0033 - loss: 0.0166 - regression_loss: 0.0266 - regression_mae: 0.1249 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0188 - val_loss: 0.0305 - val_regression_loss: 0.0234 - val_regression_mae: 0.1130 - learning_rate: 1.0000e-04
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0082 - loss: 0.0247 - regression_loss: 0.0330 - regression_mae: 0.1415 - val_classification_accuracy: 1.0000 - val_classification_loss: 0.0192 - val_loss: 0.0293 - val_regression_loss: 0


✅ Final Evaluation on Validation Dataset:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - classification_accuracy: 1.0000 - classification_loss: 0.0192 - loss: 0.0279 - regression_loss: 0.0174 - regression_mae: 0.1101
  ➜ Total Loss: 0.0279
  ➜ Classification Loss: 0.0192
  ➜ Regression Loss: 0.0174
  ➜ Classification Accuracy: 100.00%
  ➜ Regression MAE: 0.1101

✅ Final Evaluation on Test Dataset:
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - classification_accuracy: 1.0000 - classification_loss: 0.0193 - loss: 0.0366 - regression_loss: 0.0345 - regression_mae: 0.1298
  ➜ Total Loss: 0.0366
  ➜ Classification Loss: 0.0193
  ➜ Regression Loss: 0.0345
  ➜ Classification Accuracy: 100.00%
  ➜ Regression MAE: 0.1298
